# Variables

In [ ]:
import datetime
import logging
import json
import os
import requests

import psycopg2


In [17]:
URL_V2 = "https://api.reonomy.com/v2"
URL_V3 = "https://api.reonomy.com/v3"

In [18]:
auth_token = (
            "Bearer 3dad361e064eaceb1a7a342dcb22aa0c81aa166967edb5cc68be46328a2c2743"
        )

In [19]:
msa = "Dallas-Fort Worth-Arlington, TX"

In [20]:
headers = {
            "authority": "api.reonomy.com",
            "accept": "*/*",
            "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
            "authorization": auth_token,
            "content-type": "application/json",
            "origin": "https://app.reonomy.com",
            "referer": "https://app.reonomy.com/",
            "sec-ch-ua": '"Chromium";v="104", " Not A;Brand";v="99", "Google Chrome";v="104"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": '"macOS"',
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "same-site",
            "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36",
        } 

In [21]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_POSTGRES_PASSWORD = os.environ.get("AWS_POSTGRES_PASSWORD")

In [22]:
AWS_POSTGRES_PASSWORD

'Alex6581!'

## Connect to PG and return IDs

In [23]:
conn = psycopg2.connect(
            database="postgres",
            user="nexus_dev",
            password=AWS_POSTGRES_PASSWORD,  # Handle password with env file
            host="nexus-database.c0n44saktvjz.us-east-1.rds.amazonaws.com",
            port="5432",
        )
        # Connection details for AWS Relational Database Services Postgres Instance

In [24]:
cursor = conn.cursor()
cursor.execute("SELECT reonomy_id from landing_zone_reonomy")
fetched_ids = cursor.fetchall()
stored_ids = [id_[0] for id_ in fetched_ids]
# Get the IDs of properties that have already been fetched and stored within the S3 bucket to avoid duplicating data

conn.close()

## Functions

In [31]:
def gen_payload(min_acres: int):

        """Returns the payload for a filter request, based on the minimum amount of acres.

        This function will take the minimum acres argument and generate a related payload, including the max_building size and the rest of the payload.

        Args:
            min_acres (int): int representing the minimum acres.

        Returns:
            payload (object): JSON Payload containing the associated attributes based on the minimum acres.

        """

        max_building = 43560 * min_acres * 0.25

        payload = json.dumps(
            {
                "settings": {
                    "building_area": {"max": max_building},
                    "land_use_code": [
                        "806",
                        "0514",
                        "208",
                        "303",
                        "308",
                        "6018",
                        "311",
                        "6023",
                        "218",
                        "304",
                        "342",
                        "5005",
                        "6020",
                        "830",
                        "300",
                        "338",
                        "5000",
                        "5001",
                        "5008",
                        "5013",
                        "5018",
                        "5019",
                        "5020",
                        "6008",
                        "6016",
                        "320",
                        "6000",
                        "6015",
                        "321",
                        "5010",
                        "322",
                        "5012",
                        "323",
                        "324",
                        "5002",
                        "5006",
                        "5015",
                        "328",
                        "6007",
                        "326",
                        "6010",
                        "331",
                        "6019",
                        "349",
                        "6003",
                        "6014",
                        "334",
                        "6009",
                        "310",
                        "5016",
                        "5017",
                        "344",
                        "6006",
                        "258",
                        "207",
                        "6004",
                        "354",
                        "6011",
                        "358",
                        "5004",
                        "6024",
                        "361",
                        "6021",
                        "309",
                        "800",
                        "877",
                        "6001",
                        "6500",
                        "6505",
                        "6512",
                        "364",
                        "5003",
                        "6002",
                        "883",
                        "6013",
                        "886",
                        "899",
                    ],
                    "locations": [{"kind": "msa", "text": msa}],
                    "lot_size_acres": {"min": min_acres},
                    "sort": [{"name": "year_renovated", "order": "desc"}],
                },
                "aggregations": [{"name": "geo_bounds"}],
            }
        )

        return payload


In [34]:
payload_list = []
for acres_size in range(1, 3):
    payload = gen_payload(acres_size)
    payload_list.append(payload)

In [35]:
payload_list[0]

'{"settings": {"building_area": {"max": 10890.0}, "land_use_code": ["806", "0514", "208", "303", "308", "6018", "311", "6023", "218", "304", "342", "5005", "6020", "830", "300", "338", "5000", "5001", "5008", "5013", "5018", "5019", "5020", "6008", "6016", "320", "6000", "6015", "321", "5010", "322", "5012", "323", "324", "5002", "5006", "5015", "328", "6007", "326", "6010", "331", "6019", "349", "6003", "6014", "334", "6009", "310", "5016", "5017", "344", "6006", "258", "207", "6004", "354", "6011", "358", "5004", "6024", "361", "6021", "309", "800", "877", "6001", "6500", "6505", "6512", "364", "5003", "6002", "883", "6013", "886", "899"], "locations": [{"kind": "msa", "text": "Dallas-Fort Worth-Arlington, TX"}], "lot_size_acres": {"min": 1}, "sort": [{"name": "year_renovated", "order": "desc"}]}, "aggregations": [{"name": "geo_bounds"}]}'

In [42]:
current_payload = payload_list[0]

url = URL_V2 + "/search/pins?offset=0&limit=1000"    
res = requests.post(url, headers=headers, data=current_payload)
    


In [46]:
filterered_properties = res.json()['items']

In [47]:
n = 50
split_filterered_properties = [
    filterered_properties[i : i + n]
    for i in range(0, len(filterered_properties), n)
]

In [60]:
for current_split in split_filterered_properties:

    current_split_ids = [current_item["id"] for current_item in current_split]

    print(
        f"There are: {len(current_split_ids)} sites in this batch"
    )

    current_split_ids_cleaned = [
        current_split_id
        for current_split_id in current_split_ids
        if current_split_id not in stored_ids
    ]
    
    if len(current_split_ids_cleaned) == 10:
        current_split_ids_cleaned_ = current_split_ids_cleaned

    print(
        f"After filtering there are {len(current_split_ids_cleaned)} sites left to extract in this batch"
    )
    
    url = URL_V2 + "/property/summaries"

    payload = json.dumps({"property_ids": current_split_ids_cleaned})

    new_res = requests.post(url, data=payload, headers=headers)
    
    print(len(new_res.json()))

There are: 50 sites in this batch
After filtering there are 7 sites left to extract in this batch
7
There are: 50 sites in this batch
After filtering there are 5 sites left to extract in this batch
5
There are: 50 sites in this batch
After filtering there are 7 sites left to extract in this batch
7
There are: 50 sites in this batch
After filtering there are 5 sites left to extract in this batch
5
There are: 50 sites in this batch
After filtering there are 5 sites left to extract in this batch
5
There are: 50 sites in this batch
After filtering there are 4 sites left to extract in this batch
4
There are: 50 sites in this batch
After filtering there are 3 sites left to extract in this batch
3
There are: 50 sites in this batch
After filtering there are 6 sites left to extract in this batch
6
There are: 50 sites in this batch
After filtering there are 7 sites left to extract in this batch
7
There are: 50 sites in this batch
After filtering there are 5 sites left to extract in this batch
5


In [61]:
summaries = new_res.json()


In [71]:
for summary in summaries:

    property_dict = {}

    property_dict["summary"] = summary
    property_dict["scraping_details"] = {}

    # check if parcels and add details
    if "parcel_shape_ids" in summary.keys():
        property_dict["scraping_details"]["shape_ids"] = summary["parcel_shape_ids"]

    property_dict["scraping_details"]["property_id"] = summary["id"]
    
    property_dict["scraping_details"]["datetime_scraped"] = datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    property_dict["shapes"] = {}

### If shape ids

In [82]:
url = URL_V2 + "/search/property-shapes"

if ("shape_ids" in property_dict["scraping_details"].keys()):
    payload = json.dumps(
                    {"shape_ids": property_dict["scraping_details"]["shape_ids"]}
                )
    
    res = requests.post(url, headers=headers, data=payload)

In [86]:
shapes = res.json()

property_dict["shapes"]["count"] = shapes["count"]
property_dict["shapes"]["items"] = shapes["items"]

#### Details

In [95]:
url = (URL_V2 + f"/property/{property_dict['scraping_details']['property_id']}")

payload = json.dumps({})

res = requests.get(url, headers=headers)

In [98]:
details = res.json()
property_dict["details"] = details

#### Owners

In [101]:
url = (URL_V3
    + f"/property-contacts/{property_dict['scraping_details']['property_id']}"
)

res = requests.get(url, headers=headers)

In [104]:
owners = res.json()

property_dict["property_contacts"] = {}

In [106]:
if "contacts" in owners.keys():
    print("1")

    contacts = owners["contacts"]

    property_dict["property_contacts"]["contacts"] = contacts

else:  # If there is no company then there is a individual owner
    print("2")
    if "owners" in owners.keys():
        property_dict["property_contacts"]["individual_contact"] = {}
        owner = owners["owners"][0]
        property_dict["property_contacts"]["individual_contact"][
            "owner"
        ] = owner

1


In [108]:
 if "owners" in owners.keys():
        print("1")
        property_dict["property_contacts"]["owner_id"] = owners["owners"][0]["id"]

if "individual_contact" in property_dict["property_contacts"].keys():
    print("2")
    property_dict["property_contacts"]["is_contacts"] = False
    property_dict["property_contacts"]["is_individual_contact"] = True
elif "contacts" in property_dict["property_contacts"]:
    property_dict["property_contacts"]["is_individual_contact"] = False
    property_dict["property_contacts"]["is_contacts"] = True
else:  # Skip contacts
    property_dict["property_contacts"]["is_individual_contact"] = False
    property_dict["property_contacts"]["is_contacts"] = False

1


In [111]:
if property_dict["property_contacts"]["is_individual_contact"]:
    print(1)
    owner_contact_list_ids = [
                property_dict["property_contacts"]["individual_contact"]["owner"]["id"]
            ]
elif property_dict["property_contacts"]["is_contacts"]:
    print(2)
    owner_contact_list_ids = [
        contact["id"]
        for contact in property_dict["property_contacts"]["contacts"]
    ]
else:
    print(3)
    owner_contact_list_ids = []

2


In [116]:
url = URL_V3 + "/people/bulk"
if len(owner_contact_list_ids) > 0:
    

    owner_contact_list_ids = owner_contact_list_ids[
        :99
    ]  # Request only allows 100 ids to be parsed

    payload = json.dumps({"ids": owner_contact_list_ids})
    res = requests.post(url, headers=headers, data=payload)

#### Owner contacts

In [118]:
owner_contacts = res.json()

property_dict["property_contacts"]["contact_details"] = owner_contacts

In [119]:
property_dict

{'summary': {'asset_category': 'Industrial',
  'building_area': 4820.0,
  'building_update_time': '2022-09-22',
  'city': 'BURLESON',
  'fips_county': 'Tarrant County',
  'first_owner_name': 'David Quisenberry',
  'floors': 1,
  'formatted_apn': '03891461',
  'house_nbr': '12901',
  'id': '4ad0eb0f-0c0e-4719-90eb-0f0c0ee719de',
  'lot_size_acres': 1.73,
  'lot_size_sqft': 75359,
  'master_update_time': '2022-09-22',
  'mode': 'FWY',
  'mtg_update_time': '2022-09-22',
  'notes': [],
  'owner_names_count': 8,
  'owner_update_time': '2021-08-26',
  'ownership': {'green_matches_count': 1,
   'grey_matches_count': 0,
   'has_green_addresses': True,
   'has_green_emails': True,
   'has_green_phones': True,
   'has_grey_addresses': False,
   'has_grey_emails': False,
   'has_grey_phones': False},
  'parcel_shape_ids': ['1204107477', '1204197355', '1204737316'],
  'reported_owner_address': 'Po Box 1298, Burleson, TX 76097',
  'reported_owners': [{'name': 'HARRIS PARTNERSHIP'}],
  'sale_update_

### Else

In [69]:
property_dict["scraping_details"][
                    "shape_ids"
                ] = None

In [70]:
property_dict

{'summary': {'asset_category': 'Industrial',
  'building_area': 4820.0,
  'building_update_time': '2022-09-22',
  'city': 'BURLESON',
  'fips_county': 'Tarrant County',
  'first_owner_name': 'David Quisenberry',
  'floors': 1,
  'formatted_apn': '03891461',
  'house_nbr': '12901',
  'id': '4ad0eb0f-0c0e-4719-90eb-0f0c0ee719de',
  'lot_size_acres': 1.73,
  'lot_size_sqft': 75359,
  'master_update_time': '2022-09-22',
  'mode': 'FWY',
  'mtg_update_time': '2022-09-22',
  'notes': [],
  'owner_names_count': 8,
  'owner_update_time': '2021-08-26',
  'ownership': {'green_matches_count': 1,
   'grey_matches_count': 0,
   'has_green_addresses': True,
   'has_green_emails': True,
   'has_green_phones': True,
   'has_grey_addresses': False,
   'has_grey_emails': False,
   'has_grey_phones': False},
  'parcel_shape_ids': ['1204107477', '1204197355', '1204737316'],
  'reported_owner_address': 'Po Box 1298, Burleson, TX 76097',
  'reported_owners': [{'name': 'HARRIS PARTNERSHIP'}],
  'sale_update_